In [1]:
import dynet as dy

The parameter collection contains all of the learned parameters in the model.
All parameters are added to a parameter collection.

In [2]:
pc = dy.ParameterCollection()

# Parameters
First we will look at how to use Parameters in Dynet.

In [3]:
weights = pc.add_parameters((2, 4))
biases = pc.add_parameters((4,), init=dy.UniformInitializer(0.1))

`weights` is a Parameters object containing a matrix of size (2, 4). `biases` is a Parameters object containing a vector of length 4, and is initialized uniformly in [-0.1, 0.1]. Parameters are a type of Expression.

In [8]:
isinstance(weights, dy.Expression)

True

In [ ]:
weights.as_array()

In [ ]:
type(weights.as_array())

In [ ]:
biases.as_array()

Now we will perform some basic computations with these parameters. There are two important things to note. First, you should reset the computation graph by calling `dy.renew_cg()`.

First we will create a vector to pass into the network.

In [ ]:
import numpy as np
my_array_np = np.random.rand(2)
my_array_np

To put it into the computation graph, we need to call `dy.inputTensor`. This returns an input expression containing the array. It is a special type of expression that has a `set` method for setting inputs to the network.

In [ ]:
dy.renew_cg()
my_array = dy.inputTensor(my_array_np)

Then we pass it through the network by multiplying it by `weights` and adding the `biases`. 

Also note that we have to reshape the input array so that we can perform the multiplication. This adds a dimension to the vector. Before the reshape, it was a 1D vector of length 2. After, it's a 2D matrix of size 1x2. This allows us to multiply it by the 2x4 `weights` matrix. The multiplication gives us a vector of size (1, 4). So we must transpose it before adding the `biases`.

In [ ]:
my_array = dy.reshape(my_array, (1, 2))
result = my_array * weights_exp
result = dy.transpose(result)
result = result + biases_exp


Several functions allow us to investigate the `result`. `dim()` gives us the shape or dimensions of the expression and `value()` gives us the actual values in the expression. 

By the way, before calling `value()`, the computation graph hasn't actually performed any computations. Only when you request the actual numerical value of an expression does it perform the necessary computations to get the result.

In [ ]:
result.dim()

In [ ]:
result.value()

# Lookup parameters
Now we will learn how to use `dy.LookupParameters`. Generally speaking, lookup parameters are a matrix that you can select rows from using  indexing. They are more efficient to use than parameters.

In [ ]:
lookup_params = pc.add_lookup_parameters((3, 4))
lookup_params.as_array()

Lookup parameters can be used to store word vectors or feature embeddings. In this case, we have 3 rows and 4 columns, meaning that we have 3 unique vectors of size 4.

Let's look up some vectors.

In [ ]:
dy.renew_cg()
embedding_0 = lookup_params[0]
embedding_1 = lookup_params[1]
embedding_2 = lookup_params[2]

Let's verify the values.

In [ ]:
embedding_0.value()

In [ ]:
embedding_1.value()

In [ ]:
embedding_2.value()

This is what we expected (look above). Now let's show off a few things you can do in Dynet which might be useful. First, we can compute the mean of the three vectors. 

In [ ]:
embeddings_mean = dy.esum([embedding_0, embedding_1, embedding_2]) / 3.
embeddings_mean.value()